In [1]:
import tensorflow as tf


/Users/mgin/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
def wav_to_tf(input_filename):
    input_filename = tf.placeholder(tf.string, shape=[])
    output_filename = tf.placeholder(tf.string, shape=[])

    input_signal = tf.contrib.ffmpeg.decode_audio(
        tf.read_file(input_filename), file_format="wav",
        samples_per_second=44100, channel_count=2)

    

def tf_to_wav(output_filename):
    output_signal = ...  # A 2-D tensor, [samples x channels]
    encoded_audio_data = tf.contrib.ffmpeg.encode_audio(
        output_signal, file_format="wav", samples_per_second=44100)

    write_file_op = tf.write_file(output_filename, encoded_audio_data)

with tf.Session() as sess:
  sess.run(write_file_op, {input_filename: "input.wav",
                           output_filename: "output.wav"})

In [10]:
import wave
def _trivial__enter__(self):
    return self
def _self_close__exit__(self, exc_type, exc_value, traceback):
    self.close()

wave.Wave_read.__exit__ = wave.Wave_write.__exit__ = _self_close__exit__
wave.Wave_read.__enter__ = wave.Wave_write.__enter__ = _trivial__enter__

import struct
from struct import unpack

def parse_wave_python(filename):
    with wave.open(filename, 'rb') as wave_file:
        sample_rate = wave_file.getframerate()
        length_in_seconds = wave_file.getnframes() / sample_rate
        
        first_sample = struct.unpack(
            '<h', wave_file.readframes(1))[0]
        second_sample = struct.unpack(
            '<h', wave_file.readframes(1))[0]
    print('''
Parsed {filename}
-----------------------------------------------
Channels: {num_channels}
Sample Rate: {sample_rate}
First Sample: {first_sample}
Second Sample: {second_sample}
Length in Seconds: {length_in_seconds}'''.format(
            filename=filename,
            num_channels=wave_file.getnchannels(),
            sample_rate=wave_file.getframerate(),
            first_sample=first_sample,
            second_sample=second_sample,
            length_in_seconds=length_in_seconds))
        
parse_wave_python('../data/Kick_Drums/080-BD.wav')


Parsed ../data/Kick_Drums/080-BD.wav
-----------------------------------------------
Channels: 1
Sample Rate: 11025
First Sample: 258
Second Sample: 796
Length in Seconds: 0


In [13]:
import tensorflow as tf
# Requires latest tf-1.4 on Windows
from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
def parse_wave_tf(filename):
    audio_binary = tf.read_file(filename)
    desired_channels = 1
    wav_decoder = contrib_audio.decode_wav(
        audio_binary,
        desired_channels=desired_channels)
    with tf.Session() as sess:
        sample_rate, audio = sess.run([
            wav_decoder.sample_rate,
            wav_decoder.audio])
        first_sample = audio[0][0] * (1 << 15)
        second_sample = audio[1][0] * (1 << 15)
        print('''
Parsed {filename}
-----------------------------------------------
Channels: {desired_channels}
Sample Rate: {sample_rate}
First Sample: {first_sample}
Second Sample: {second_sample}
Length in Seconds: {length_in_seconds}'''.format(
            filename=filename,
            desired_channels=desired_channels,
            sample_rate=sample_rate,
            first_sample=first_sample,
            second_sample=second_sample,
            length_in_seconds=len(audio) / sample_rate))
        
        # save as tfrecord
        
        
parse_wave_tf('../data/Kick_Drums/VEH3 Kicks 220.wav')


Parsed ../data/Kick_Drums/VEH3 Kicks 220.wav
-----------------------------------------------
Channels: 1
Sample Rate: 44100
First Sample: 9962.0
Second Sample: 14665.0
Length in Seconds: 0


In [25]:
# apply noise

import numpy as np
from scikits.audiolab import wavread, wavwrite

data1, fs1, enc1 = wavread("../data/Kick_Drums/VEH3 Kicks 220.wav")
data2, fs2, enc2 = wavread("../data/Kick_Drums/VEH3 Kicks 221.wav")

assert fs1 == fs2
assert enc1 == enc2

# take min size
#min_size = min(len(data1), len(data2))
#result = 2 * data1[:min_size] + 2 * data2[:min_size]

#result = 0.5 * data1 + 0.5 * data2

# wrap to longest
short, long = (data1, data2) if len(data1) < len(data2) else (data2, data1)
n = len(long) / len(short)
new_array = np.tile(short, n)

result = 5 * long[:n] + 5 * new_array

wavwrite(result, 'result.wav')